<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Bayesian Linear Mixed Models (Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [1]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [2]:
phenofile  = Datasets.dataset("example","phenotypes.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")
genofile   = Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true,missingstrings=["NA"])
pedigree   = get_pedigree(pedfile,separator=",",header=true);

The delimiter in pedigree.txt is ','.
Pedigree informatin:
#individuals: 12
#sires:       4
#dams:        5
#founders:    3


In [3]:
first(phenotypes,5)

,ID,y1,y2,y3,x1,x2,x3,dam,weights
,String,Float64⍰,Float64⍰,Float64⍰,Float64,Float64,String,String⍰,Float64
1,a1,-0.06,3.58,-1.18,0.9,2.0,m,missing,1.0
2,a3,-2.07,3.19,1.0,0.7,2.0,f,missing,1.0
3,a4,-2.63,6.97,-0.83,0.6,1.0,m,a2,1.0
4,a5,2.31,missing,-1.52,0.4,2.0,m,a2,1.0
5,a7,missing,missing,missing,0.1,0.1,m,a3,1.0


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univariate Linear Mixed Model (Genomic data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [4]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID + dam";

In [5]:
model1 = build_model(model_equation1);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [6]:
set_covariate(model1,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [7]:
set_random(model1,"x2");
set_random(model1,"ID dam",pedigree);

<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [8]:
add_genotypes(model1,genofile,separator=',');

The delimiter in genotypes.txt is ','.
The header (marker IDs) is provided in genotypes.txt.
5 markers on 7 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [9]:
out1=runMCMC(model1,phenotypes,methods="BayesC",estimatePi=true);

Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Phenotypes for all traits included in the model for individual a7 in the row 5 are missing. This record is deleted.
The number of observations with both genotypes and phenotypes used in the analysis is 4.
The number of observations with both phenotype and pedigree information used in the analysis is 4.
Prior information for genomic variance is not provided and is generated from the data.
Prior information for residual variance is not provided and is generated from the data.
Prior information for random effect variance is not provided and is generated from the data.
Prior information for random effect variance is not provided and is generated from the data.
Missing values are found in independent variables: dam.

The prior for marker effects variance is calculated from the genetic variance and π.
The mean of the prior for the marker effects variance is: 0.617255



A Linear Mixed Mod

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:02




The version of Julia and Platform in use:

Julia Version 1.3.1
Commit 2d5741174c (2019-12-30 21:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [10]:
keys(out1)

Base.KeySet for a Dict{Any,Any} with 9 entries. Keys:
  "marker effects"
  "EBV_y1"
  "Pi"
  "heritability"
  "location parameters"
  "residual variance"
  "polygenic effects covariance matrix"
  "genetic_variance"
  "marker effects variance"

In [11]:
out1["EBV_y1"]

,ID,EBV,PEV
,Any,Any,Any
1,a1,0.446483,1.80448
2,a3,0.246589,1.93395
3,a4,-1.20395,1.3241
4,a5,-0.334794,1.48462
5,a6,0.239163,2.05027
6,a7,0.532515,2.21422
7,a8,-0.431805,1.82995


In [12]:
out1["Pi"]

,π,Estimate,Std_Error
,Any,Any,Any
1,π,0.427833,0.279688


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Mixed Model (Genomic data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [13]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [14]:
model2 = build_model(model_equation2);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [15]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [16]:
set_random(model2,"x2");
set_random(model2,"ID dam",pedigree);

x2 is not found in model equation 1.
dam is not found in model equation 2.
dam is not found in model equation 3.


<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [17]:
add_genotypes(model2,genofile,separator=',');

The delimiter in genotypes.txt is ','.
The header (marker IDs) is provided in genotypes.txt.
5 markers on 7 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [18]:
out2=runMCMC(model2,phenotypes,methods="BayesC",estimatePi=true);

Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Phenotypes for all traits included in the model for individual a7 in the row 5 are missing. This record is deleted.
The number of observations with both genotypes and phenotypes used in the analysis is 4.
The number of observations with both phenotype and pedigree information used in the analysis is 4.
Prior information for genomic variance is not provided and is generated from the data.
Prior information for residual variance is not provided and is generated from the data.
Prior information for random effect variance is not provided and is generated from the data.
Prior information for random effect variance is not provided and is generated from the data.
Missing values are found in independent variables: dam.

Pi (Π) is not provided.
Pi (Π) is generated assuming all markers have effects on all traits.

The prior for marker effects covariance matrix is calculated from genetic covar

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:03




The version of Julia and Platform in use:

Julia Version 1.3.1
Commit 2d5741174c (2019-12-30 21:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i7-7700K CPU @ 4.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, skylake)


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [19]:
keys(out2)

Base.KeySet for a Dict{Any,Any} with 11 entries. Keys:
  "location parameters"
  "polygenic effects covariance matrix"
  "EBV_y3"
  "genetic_variance"
  "EBV_y2"
  "EBV_y1"
  "marker effects"
  "heritability"
  "residual variance"
  "Pi"
  "marker effects variance"

In [20]:
out2["Pi"]

,π,Estimate,Std_Error
,Any,Any,Any
1,"[1.0, 1.0, 0.0]",0.136707,0.118103
2,"[0.0, 0.0, 0.0]",0.135537,0.110178
3,"[1.0, 0.0, 0.0]",0.147073,0.118746
4,"[0.0, 1.0, 1.0]",0.110508,0.0945137
5,"[1.0, 0.0, 1.0]",0.108752,0.117394
6,"[0.0, 0.0, 1.0]",0.109641,0.10215
7,"[1.0, 1.0, 1.0]",0.129829,0.117483
8,"[0.0, 1.0, 0.0]",0.121955,0.106232


In [21]:
out2["EBV_y2"]

,ID,EBV,PEV
,Any,Any,Any
1,a1,-1.15508,0.718204
2,a3,0.657493,2.40265
3,a4,-0.586396,1.05399
4,a5,-0.65314,1.99496
5,a6,-0.164568,1.64079
6,a7,-0.723695,1.48565
7,a8,-0.814014,2.1087
